In [47]:
import pandas as pd
import os

import math
from collections import Counter

In [39]:

Data=pd.read_csv('./Experiment_Attendance.txt')

In [15]:
def get_Day(date):
    Days=['월','화','수','목','금','토','일']
    M=int(date[0:2])
    D=int(date[2:])
    if M==11: D+=31
    if M==12: D+=61
    return Days[(D-22)%7];

In [101]:
def get_elementwise_sim(routine1,routine2):
    sim=0
    for i in range(len(routine1)):
        for j in routine1[i].keys():
            sim+=routine2[i][j]*routine1[i][j]
    return sim/len(routine1)

# 2. Sequence화 ( 144 Seq for One day)
User * Date * Seq[144] 

0:0~10, 1:10:01~11:00, ... 

Row Data : 2018-10-22 08:50:58,GWANJUNG1,SC00062,-85 => 530:58 => [53]

In [83]:
Day_Weight={'월':1,'화':1,'수':1,'목':1,'금':1,'토':10,'일':10}
Categories=dict()
Categories={
    '33-331':1,'33-330':1,'33-328':1,'33-327':1,
    '33-231':1,'33-230':1,'33-229':1,'33-228':1,'33-226':1,'33-225':1,
    '33-B1L':3,
    '43-1':4,
    'BONGUS':5,
    'MugOut':6,'Muginside':6,
    'Gym':7,
    'HH1':8,'HH2':8,'HH3':8,
    'GWANJUNG1':9,'Gwanjung2':9,'GWANJUNG3':9,'Gwanjung3':9,'GWANJUNG1':9,
    'CU':10,
    'BBQ':11,
    'Sinyang3':12,
    'Doodam3':13,
    'Doodam4':14,
    'Doodam':15,
    'Haedong':16
}

In [68]:
timegaps=600 # 10 Minutes 
user_daily_routine=[]
for i in range(100):
    user_daily_routine.append([])
    for j in range(50):
        user_daily_routine[-1].append([])
        for k in range(int(86400/timegaps)+1):
            user_daily_routine[-1][-1].append(0)

all_files=os.listdir("./Raw/")
txt_files=[];
for X in all_files:
    if X[-4:]=='.txt':
        txt_files.append(X);
print(len(txt_files))  # only text files

48


In [69]:
X=extract_routine('./Raw/1022dump.txt','SC00094',600)
X

[Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),
 Counter({1: 1.0}),


In [76]:
def extract_routine(source,S,Interval):
    length=int(86400/Interval)
    routine=[]
    for i in range(length+1):
        routine.append(Counter())
    for X in open(source,'r').readlines():
        if S not in X : continue
        place=X.split(',')[1]
        hour,minute,second=map(int,X.split(',')[0].split(' ')[1].split(':'))
        try:
            routine[math.ceil((hour*3600+minute*60+second)/Interval)][Categories[place]]+=1
        except:
            print(X)
    # routine : Counter의 집합, 각 Counter : [Category, 신호수]
    # TODO : [1:80% 2,20%] VS [1:20%, 3:60%, 2:20%] -> 0.2+0.2 = 0.4 or 
    # 맥스로 하면 좋은점: 쉬움
    # 확률곱으로 하면 좋은점 : 좀더 자세함/ 한친구가 늦게오면? 확률곱이 더 좋은데 
    # 확률곱으로 하자! (행동패턴이 복잡한것도 캐치가능 Ex 수업->봉구스->수업)
    normalized_routine=[]
    for t in routine:
        total=sum(t.values())
        for x in t.keys():
            t[x]=t[x]/total
        normalized_routine.append(t)
    return normalized_routine
    

> Read 하고 10분간의 데이터 모아서 가장 높은 애로 선택(Counter) 하기

> Seq Index로 Iter, Counter 만들어서 카테고리 값 추가

In [79]:
Days=[]
txt_files.sort()
for Target in range(10,99):
    S='SC000'+str(Target)
    for i in range(len(txt_files)):
        if Target==10: Days.append(get_Day(txt_files[i][0:4]))
#        print(txt_files[i],get_Day(txt_files[i][0:4]))
        user_daily_routine[Target][i]=extract_routine('./Raw/'+txt_files[i],S,timegaps)


In [103]:
T=15

for i in range(10,99):
    for j in range(i+1,99):
        sim=get_elementwise_sim(user_daily_routine[i][T],user_daily_routine[j][T])
        if sim!=0: 
            print("%d between %d : %lf" %(i,j,get_elementwise_sim(user_daily_routine[i][T],user_daily_routine[j][T])))
    break

10 between 14 : 0.027586
10 between 15 : 0.071141
10 between 17 : 0.096552
10 between 18 : 0.003161
10 between 22 : 0.020690
10 between 23 : 0.003161
10 between 26 : 0.089655
10 between 28 : 0.089655
10 between 29 : 0.045436
10 between 31 : 0.089655
10 between 32 : 0.041379
10 between 33 : 0.052896
10 between 34 : 0.027586
10 between 35 : 0.060064
10 between 36 : 0.016804
10 between 37 : 0.029023
10 between 38 : 0.027586
10 between 42 : 0.092619
10 between 43 : 0.000208
10 between 44 : 0.048440
10 between 45 : 0.027586
10 between 46 : 0.022473
10 between 47 : 0.026820
10 between 48 : 0.027586
10 between 53 : 0.033251
10 between 56 : 0.089655
10 between 57 : 0.027586
10 between 58 : 0.006897
10 between 60 : 0.059306
10 between 61 : 0.003502
10 between 62 : 0.027586
10 between 63 : 0.056399
10 between 64 : 0.060338
10 between 66 : 0.068719
10 between 67 : 0.066980
10 between 68 : 0.061741
10 between 69 : 0.056426
10 between 70 : 0.040802
10 between 71 : 0.064312
10 between 72 : 0.061928


In [85]:
Day_Weight[Days[15]]

1